In [1]:
import pandas as pd
pd.set_option("chained_assignment", None)
pd.set_option("display.max_rows", 5000)
pd.set_option("display.max_columns", 5000)
pd.set_option("display.width", 10000)


df = pd.read_parquet("data_for_anton.parquet")
print(df)
df_formatted=pd.DataFrame(columns=["benchmark","prior","algorithm","fidelity","seed","score"])
df=df.reset_index()
df_collection=[]
for seed_nr in range(50):
    partial_df=df[["benchmark","prior","algorithm","used_fidelity"]]
    partial_df["value"]=df[f"seed-{seed_nr}"]
    partial_df["seed"]=seed_nr
    df_collection.append(partial_df)
    print(f"Seed {seed_nr+1}/50", end="\r", flush=True)
df_formatted=pd.concat(df_collection,ignore_index=True)

                                                                                   seed-0    seed-1    seed-2    seed-3    seed-4    seed-5    seed-6    seed-7    seed-8    seed-9   seed-10   seed-11   seed-12   seed-13   seed-14   seed-15   seed-16   seed-17   seed-18   seed-19   seed-20   seed-21   seed-22   seed-23   seed-24   seed-25   seed-26   seed-27   seed-28   seed-29   seed-30   seed-31   seed-32   seed-33   seed-34   seed-35   seed-36   seed-37   seed-38   seed-39   seed-40   seed-41   seed-42   seed-43   seed-44   seed-45   seed-46   seed-47   seed-48   seed-49
benchmark      prior algorithm                                    used_fidelity                                                                                                                                                                                                                                                                                                                                                     

In [2]:
unique=list(df_formatted["algorithm"].unique())
dict={}
for algo in df_formatted["algorithm"].unique():
    if "pb_mutation" in algo:
        dict[algo]="pb_mutation"
        unique.remove(algo)
    elif "priorband_hypersphere" in algo or "pb_hypersphere" in algo:
        dict[algo]="priorband_hypersphere"
        unique.remove(algo)
    elif "priorband_crossover" in algo or "pb_crossover" in algo:
        dict[algo]="priorband_crossover"
        unique.remove(algo)
    elif "random_search" in algo:
        dict[algo]="random_search"
        unique.remove(algo)
    elif "bo" in algo:
        dict[algo]="bo"
        unique.remove(algo)
    elif "asha" in algo:
        dict[algo]="asha"
        unique.remove(algo)
    elif "hb" in algo or "hyperband" in algo:
        dict[algo]="hb"
        unique.remove(algo)
print(dict)

def map_values(value):
    mapped_value = dict.get(value, value)
    return mapped_value

df_formatted["algorithm_family"]=df_formatted['algorithm']
print(len(df_formatted["algorithm_family"].unique()))
df_formatted["algorithm_family"]=df_formatted['algorithm_family'].map(map_values)
print(df_formatted["algorithm_family"].unique())

{'pb_mutation_dynamic_linear-default-at-target': 'pb_mutation', 'priorband_crossover_decay_linear': 'priorband_crossover', 'pb_mutation_decay_geometric_bo-no-default': 'pb_mutation', 'pb_mutation_dynamic_linear_bo': 'pb_mutation', 'pibo-no-default': 'bo', 'pb_mutation_constant_linear': 'pb_mutation', 'pb_hypersphere_dynamic_50': 'priorband_hypersphere', 'pb_mutation_constant_50': 'pb_mutation', 'priorband_hypersphere_dynamic-default-at-target': 'priorband_hypersphere', 'pb_mutation_decay_geometric-no-default': 'pb_mutation', 'bo': 'bo', 'bohb': 'bo', 'hb_inc': 'hb', 'pb_mutation_decay_geometric-default-at-target': 'pb_mutation', 'asha_hb_priorband': 'asha', 'hyperband_prior-no-default': 'hb', 'bo-10': 'bo', 'pb_mutation_decay_geometric_bo-default-at-target': 'pb_mutation', 'priorband_crossover_constant_linear': 'priorband_crossover', 'priorband_crossover_decay': 'priorband_crossover', 'pb_mutation_dynamic_linear': 'pb_mutation', 'pb_mutation_dynamic_geometric-no-default': 'pb_mutation'

In [3]:
from pymer4 import Lmer
from scipy import stats

df_model=df_formatted#.loc[(df_formatted["algorithm_family"]=="pb_mutation")|(df_formatted["algorithm_family"]=="random_search")]

def create_model(system_id:str="algorithm",fidelity_type:int=0):
    metric="value"
    input_id="benchmark"
    fidelity="used_fidelity"
    df_model[input_id]=df_model[input_id].astype(str)
    df_model[system_id]=df_model[system_id].astype(str)
    df_model[metric]=df_model[metric].astype(float)
    df_model[fidelity]=df_model[fidelity].astype(float)

    
    if fidelity_type==0:
        model = Lmer(
            formula=f"{metric}~{system_id} + (1|{input_id})", data=df_model
        )
    elif fidelity_type== 1:
        model = Lmer(
            formula=f"{metric}~{system_id} + (1|{fidelity})+ (1|{input_id})", data=df_model
        )
    elif fidelity_type== 2:
        model = Lmer(
            formula=f"{metric}~{system_id} + {fidelity}+ (1|{input_id})", data=df_model
        )
    model.fit(
        factors={system_id: list(df_model[system_id].unique())},
        REML=False,
        summarize=False,
        verbose=True)
    return model

def glrt(mod1: Lmer, mod2: Lmer):
    chi_square = 2 * abs(mod1.logLike - mod2.logLike)
    delta_params = abs(len(mod1.coefs) - len(mod2.coefs))
    return {
        "chi_square": chi_square,
        "df": delta_params,
        "p": 1 - stats.chi2.cdf(chi_square, df=delta_params),
    }



In [4]:
linear_fidelity_model_fam=create_model("algorithm_family",fidelity_type=1)
grouped_fidelity_model_fam=create_model("algorithm_family",fidelity_type=2)

Fitting linear model using lmer with Wald confidence intervals...

**NOTE**: Column for 'residuals' not created in model.data, but saved in model.resid only. This is because you have rows with NaNs in your data.

**NOTE** Column for 'fits' not created in model.data, but saved in model.fits only. This is because you have rows with NaNs in your data.

Fitting linear model using lmer with Wald confidence intervals...

**NOTE**: Column for 'residuals' not created in model.data, but saved in model.resid only. This is because you have rows with NaNs in your data.

**NOTE** Column for 'fits' not created in model.data, but saved in model.fits only. This is because you have rows with NaNs in your data.



print(no_fidelity_model_algo.logLike)
print(linear_fidelity_model_algo.logLike)
print(grouped_fidelity_model_algo.logLike)

print("GLRT: No fidelity vs. grouped:")
print(glrt(no_fidelity_model_algo,grouped_fidelity_model_algo),end="\n")
print("GLRT: No fidelity vs. linear:")
print(glrt(no_fidelity_model_algo,linear_fidelity_model_algo),end="\n")
print("GLRT: Grouped fidelity vs. linear:")
print(glrt(linear_fidelity_model_algo,grouped_fidelity_model_algo),end="\n")

print(no_fidelity_model_fam.logLike)
print(linear_fidelity_model_fam.logLike)
print(grouped_fidelity_model_fam.logLike)

print("GLRT: No fidelity vs. grouped:")
print(glrt(no_fidelity_model_fam,grouped_fidelity_model_fam),end="\n")
print("GLRT: No fidelity vs. linear:")
print(glrt(no_fidelity_model_fam,linear_fidelity_model_fam),end="\n")
print("GLRT: Grouped fidelity vs. linear:")
print(glrt(linear_fidelity_model_fam,grouped_fidelity_model_fam),end="\n")